In [12]:
import pandas as pd
import openai
from dotenv import load_dotenv
import os

# Load environment variables from .env file
env_path = "C:\\Users\\wongb\\user_trajectories_winter\\494-user-trajectories\\students\\benedict\\z\\.env"
load_dotenv(dotenv_path=env_path)

# Function to classify tweets based on partisan lean
def classify_tweets(csv_path):
    # Load the CSV file
    df = pd.read_csv(csv_path)
    
    # Check if 'tweet' column exists
    if 'tweet' not in df.columns:
        raise ValueError("The CSV must contain a 'tweet' column.")
    
    # Initialize OpenAI client with API key from environment variable
    client = openai.OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
    
    # Function to get partisan lean from OpenAI
    def get_partisan_lean(tweet):
        prompt = f"""Classify the following tweet as LEFT, RIGHT, or NEUTRAL based on the ideological disposition of the speaker/writer.

DEFINITIONS:
- LEFT: Any position from liberal democratic to communist, and anything in between. Includes liberal democrat, progressive, socialist, social democratic, anti-colonialist and anti-imperialist, anarchist, Marxist, internationalist, and communist ideologies.
- RIGHT: Any position from liberal conservative to fascist, and anything in between. Includes conservative, libertarian-right, authoritarian-right, monarchist, nationalist, anti-woke, religious conservatism, anti-globalism, racist, sexist, and fascist ideologies.
- NEUTRAL: Unopinionated, objective statements such as news reporting, scientific statements, factual observations, commonly held apolitical opinions or statements, or purely informational or advertising content with no ideological framing. However, tone and reason for pointing certain events out must be checked to see if the writer views this news as good or not potentially indicating a partisan leaning.

IMPORTANT GUIDANCE:
Judge based on the ideological disposition of the speaker/writer, not based on a checklist of specific positions. Take into account internet style subcultures, memes, coded language, and subcultural references that may convey ideological meaning beyond surface-level keywords. Consider tone, framing, and the broader context of online discourse. Answer the question based on who would write this?

Tweet to classify: {tweet}
Return ONLY ONE of the following words as your answer: LEFT, RIGHT, NEUTRAL. Do not provide any additional explanation or text.
"""
        response = client.chat.completions.create(
            model='gpt-4o-mini',
            messages=[{'role': 'user', 'content': prompt}]
        )
        return response.choices[0].message.content
    
    # Apply the classification to each tweet
    df['llm_label'] = df['tweet'].apply(get_partisan_lean)
    
    return df


df = classify_tweets('C:\\Users\\wongb\\user_trajectories_winter\\494-user-trajectories\\students\\benedict\\data\\mitweet_sample_BW_labeled.csv')
df


,topic,tweet,partisan_lean,human_label,llm_label
0,Black Lives Matter,#BlackLivesMatter ride to #Ferguson has left m...,LEFT,LEFT,LEFT
1,Women’s Right,Oh if you have Disney+ I really recommend the ...,LEFT,LEFT,LEFT
2,Political Parties,This was no mistake. This was a calculated dog...,CENTER,LEFT,LEFT
3,Abortion,Biden still seems to think the Republican Part...,RIGHT,LEFT,LEFT
4,Political Parties,Never thought I would live in a country where ...,LEFT,RIGHT,RIGHT
...,...,...,...,...,...
95,Russo-Ukrainian War,Levelling-up' is a scam. Cancelling #Netflix w...,LEFT,LEFT,LEFT
96,Abortion,New: Texas can enforce its pre-Roe abortion ba...,MIXED,NEUTRAL,NEUTRAL
97,Russo-Ukrainian War,Will join \r\n and \r\n on \r\n at 9: 35 PM ES...,CENTER,NEUTRAL,NEUTRAL
98,Black Lives Matter,"""A white girl had to die for people to pay att...",LEFT,LEFT,LEFT


In [13]:
df.to_csv('C:\\Users\\wongb\\user_trajectories_winter\\494-user-trajectories\\students\\benedict\\data\\mitweet_sample_BW_labeled_with_llm.csv', index=False)

In [14]:
import warnings
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import pandas as pd

# Suppress UndefinedMetricWarning
warnings.filterwarnings('ignore', category=UserWarning)

# Normalize partisan_lean: map CENTER and MIXED to NEUTRAL
partisan_lean_normalized = df['partisan_lean'].replace(['CENTER', 'MIXED'], 'NEUTRAL')

def get_metrics(y_true, y_pred, model_name):
    """Return metrics as a DataFrame with overall and per-label statistics"""
    
    # Calculate overall metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
    
    # Get confusion matrix and labels
    labels = sorted(set(y_true) | set(y_pred))
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    
    # Calculate per-label accuracy (diagonal / row sum)
    per_label_acc = {}
    for i, label in enumerate(labels):
        total = cm[i].sum()
        correct = cm[i, i]
        per_label_acc[label] = correct / total if total > 0 else 0
    
    # Create per-label accuracy string
    per_label_str = ', '.join([f"{label}: {per_label_acc[label]:.2%}" for label in labels])
    
    # Create metrics row
    metrics_df = pd.DataFrame({
        'Model': [model_name],
        'Accuracy': [f"{accuracy:.2%}"],
        'Precision': [f"{precision:.2%}"],
        'Recall': [f"{recall:.2%}"],
        'F1-Score': [f"{f1:.2%}"],
        'Per-Label Accuracy': [per_label_str]
    })
    
    return metrics_df

# Get metrics for both models
llm_metrics = get_metrics(df['human_label'], df['llm_label'], "LLM Label")
partisan_metrics = get_metrics(df['human_label'], partisan_lean_normalized, "Partisan Lean")

# Combine into one table
pd.concat([llm_metrics, partisan_metrics], ignore_index=True)

,Model,Accuracy,Precision,Recall,F1-Score,Per-Label Accuracy
0,LLM Label,83.00%,82.96%,83.00%,82.86%,"LEFT: 89.83%, NEUTRAL: 73.68%, RIGHT: 72.73%"
1,Partisan Lean,41.00%,53.06%,41.00%,43.59%,"LEFT: 40.68%, NEUTRAL: 52.63%, RIGHT: 31.82%"
